In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# Keys to csv mapping
case_code = {'Cmpgaze_V': 'Composite Gaze - With Vision',
             'DH_L' : 'Dix-Hallpike - Head Left - Vision Denied',
             'DH_R' : 'Dix-Hallpike - Head Right - Vision Denied',
             'GazeC_V' : 'Gaze - Center - With Vision',
             'GazeC_VD' : 'Gaze - Center - Vision Denied',
             'GazeD_V' : 'Gaze - Down - With Vision',
             'GazeD_VD' : 'Gaze - Down - Vision Denied',
             'GazeU_V' : 'Gaze - Up - With Vision',
             'GazeU_VD' : 'Gaze - Up - Vision Denied',
             'GazeL_V' : 'Gaze - Left - With Vision',
             'GazeL_VD' : 'Gaze - Left - Vision Denied',
             'GazeR_V' : 'Gaze - Right - With Vision',
             'GazeR_VD' : 'Gaze - Right - Vision Denied',
             'GazeReb_R_VD' : 'Gaze - Rebound Right - Vision Denied',
             'GazeReb_R_V' : 'Gaze - Rebound Right - With Vision',
             'GazeReb_L_VD' : 'Gaze - Rebound Left - Vision Denied',
             'GazeReb_L_V' : 'Gaze - Rebound Left - With Vision',
             'HSN' : 'Headshake - Head Straight - Vision Denied',
             'Saccade' : '-Laser Saccade',
             'Bow' : 'Lateropulsion-Bow-Lean - Chin to Chest - Vision Denied',
             'lean' : 'Lateropulsion-Bow-Lean - Head Extended Backward - Vision Denied',
             'Bow/Lean_V' : 'Lateropulsion-Bow-Lean - Sitting - With Vision',
             'Roll_L' : 'Roll - Leftward - Vision Denied',
             'Roll_R' : 'Roll - Rightward - Vision Denied'
}

def fix_key(k):
    if '/' in k:
        k = k.replace('/','')
    return k

def fix_value(v):
    if ' ' in v:
        v = v.replace(' ', '')
    v = v.lower()
    return v

def remake_keys(key_value):
    name_code =  {fix_value(value):fix_key(key) for key, value in key_value.items()}
    return name_code

name_code = remake_keys(case_code)

In [3]:
# helper for reordering segs by timestamp
def ind(x):
    return int(x.split('_')[-1].split('.')[0])

In [4]:
def find_centroid(fp):
    seg = np.load(fp)
    seg[np.where(seg == 1.0)] = 0
    seg[np.where(seg == 2.0)] = 0
    pupil_coords = np.argwhere(seg == 3.0)
    centroid = np.mean(pupil_coords,axis=0)
    centroid_fixed = (centroid[1], seg.shape[0]-centroid[0])
    return centroid_fixed

In [5]:
def find_centroids(npyfiles):
    centroids = []
    consecutive10 = 0
    for fp in npyfiles:
        centroid = find_centroid(fp)
        if centroid[0] != centroid[0]:
            #raise Exception(fp)
            consecutive10 += 1
        elif centroid[1] != centroid[1]:
            #raise Exception(fp)
            consecutive10 += 1
        else:
            consecutive10 = 0
        if consecutive10 > 10:
            raise Exception(fp)
        centroids.append(centroid)
    return centroids

In [6]:
seg_dir = '../data/images/seg/'

In [7]:
%%time
samples = 0
columns = ['Video'] + [i for i in range(1,601)]
data = []
fails = 0
for case in os.listdir(seg_dir):
    case_path = seg_dir + case + '/'
    for vid in os.listdir(case_path):
        vid_path = case_path + vid + '/'
        vid_key = vid.replace(' ', '').lower()
        name = case + '-' + name_code[vid_key]
        if len(os.listdir(vid_path)) == 1200:
            files = [vid_path + file for file in os.listdir(vid_path)]
            npyfiles = sorted(list(filter(lambda x: 'npy' in x, files)), key=ind)
            #print('Processing ', name + ' ...')
            try:
                row = [name] + find_centroids(npyfiles)
                data.append(row)
                samples += 1
            except:
                print('Failed Video:', name)
                fails += 1

/home-3/nwagle1@jhu.edu/.conda/envs/aeye/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home-3/nwagle1@jhu.edu/.conda/envs/aeye/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Failed Video: Case_12-HSN
Failed Video: Case_12-Cmpgaze_V
Failed Video: Case_12-Roll_R
Failed Video: Case_12-DH_L
Failed Video: Case_12-DH_R
Failed Video: Case_12-lean
Failed Video: Case_12-Roll_L
Failed Video: Case_12-GazeD_VD
Failed Video: Case_12-BowLean_V
Failed Video: Case_18-GazeL_VD
Failed Video: Case_18-HSN
Failed Video: Case_18-DH_R
Failed Video: Case_18-Bow
Failed Video: Case_18-DH_L
Failed Video: Case_18-lean
Failed Video: Case_18-Roll_L
Failed Video: Case_18-BowLean_V
Failed Video: Case_20-DH_L
Failed Video: Case_20-BowLean_V
Failed Video: Case_20-GazeL_V
Failed Video: Case_20-HSN
Failed Video: Case_20-Roll_R
Failed Video: Case_20-Roll_L
Failed Video: Case_20-GazeL_VD
Failed Video: Case_20-GazeR_VD
Failed Video: Case_20-DH_R
Failed Video: Case_01-Roll_L
Failed Video: Case_01-GazeReb_R_V
Failed Video: Case_01-DH_R
Failed Video: Case_01-Roll_R
Failed Video: Case_01-DH_L
Failed Video: Case_01-GazeD_V
Failed Video: Case_01-BowLean_V
Failed Video: Case_01-GazeL_VD
Failed Video: 

In [8]:
df = pd.DataFrame(data, columns=columns)
df.set_index('Video')
df.head()

,Video,1,2,3,4,5,6,7,8,9,...,591,592,593,594,595,596,597,598,599,600
0,Case_12-GazeL_VD,"(228.4360902255639, 138.22481203007519)","(227.82686335403727, 137.41149068322983)","(227.6813432835821, 137.68880597014925)","(227.20451237263464, 137.5524017467249)","(226.9862914862915, 137.44227994227992)","(226.74466519499632, 137.00220750551875)","(226.59818308874912, 137.70300489168415)","(226.64752333094043, 137.1608040201005)","(226.39112050739956, 137.22339675828047)",...,"(214.83861301369862, 142.87371575342468)","(215.47091062394603, 143.16315345699832)","(216.2292576419214, 142.70873362445417)","(215.62624434389141, 142.11447963800904)","(215.6035874439462, 142.32376681614352)","(215.3776546629732, 141.79455216989842)","(214.75184162062615, 141.60405156537752)","(214.77448071216617, 143.08266214497667)","(214.60439087387, 142.75419715884632)","(214.54987212276214, 142.98422847399831)"
1,Case_12-Bow,"(135.22558139534883, 184.00581395348837)","(135.91166077738515, 184.5053003533569)","(136.61144219308702, 184.57330154946365)","(136.83952095808382, 184.39520958083833)","(136.76555023923444, 184.1543062200957)","(136.72142857142856, 184.07738095238096)","(136.57857142857142, 184.03690476190476)","(136.29383886255926, 183.94668246445497)","(136.2158273381295, 183.9988009592326)",...,"(149.57328145265888, 183.27237354085602)","(149.43864229765012, 183.09138381201043)","(149.27190721649484, 183.05927835051546)","(149.15267175572518, 182.95801526717557)","(149.0252525252525, 183.01262626262627)","(148.68348045397227, 182.9747793190416)","(148.58080808080808, 183.00252525252526)","(148.53934010152284, 182.99111675126903)","(148.56632653061226, 182.98469387755102)","(148.60153256704982, 182.80204342273308)"
2,Case_12-GazeReb_R_V,"(67.27417302798982, 92.01653944020356)","(65.8700964630225, 90.72218649517686)","(66.5629348513599, 91.01897533206832)","(66.67504835589942, 91.38039974210187)","(66.5978120978121, 91.42149292149293)","(66.45425867507886, 91.58170347003156)","(66.31207004377737, 91.68542839274548)","(66.37104930467763, 91.73577749683943)","(66.3352417302799, 91.78944020356235)",...,"(67.79631828978623, 93.88776722090262)","(67.6249265138154, 93.9758965314521)","(66.61474435196195, 93.23365041617123)","(67.14487632508833, 93.236160188457)","(67.34313137372526, 93.40431913617277)","(67.26698450536352, 93.43087008343267)","(67.19372037914692, 93.48281990521326)","(67.12908982748364, 93.62403331350387)","(67.12908982748364, 93.62403331350387)","(67.13565426170469, 93.70528211284514)"
3,Case_12-GazeL_V,"(216.21132075471698, 132.79433962264153)","(216.16315307057747, 132.967919340055)","(215.82825484764544, 132.9427516158818)","(215.89764492753622, 133.01177536231884)","(216.2323612417686, 133.1881467544685)","(216.80664294187426, 133.20166073546858)","(216.38577586206895, 134.01400862068965)","(215.8048245614035, 133.77083333333331)","(215.88654060066742, 133.85094549499445)",...,"(214.07957559681697, 140.31697612732097)","(213.8155981493721, 140.2088565763384)","(213.76388888888889, 140.20634920634922)","(213.6730137885752, 140.14379514116877)","(213.61502966381016, 140.1456822676335)","(214.13373253493015, 140.36526946107784)","(216.0207927225471, 140.85510071474982)","(215.9072031148605, 141.0590525632706)","(215.70672762900065, 141.18158066623124)","(215.61359093937375, 140.96735509660226)"
4,Case_12-GazeC_V,"(159.42724637681158, 124.22028985507247)","(159.10526315789474, 123.73859649122807)","(159.09335727109516, 123.43147815679234)","(159.09335727109516, 123.43147815679234)","(159.23469387755102, 123.34333733493398)","(159.32404603270746, 123.24591156874621)","(159.337003058104, 123.25321100917431)","(159.30488559059987, 123.26963512677798)","(159.3975, 123.15625)",...,"(161.96938775510205, 105.36734693877551)","(nan, nan)","(nan, nan)","(nan, nan)","(nan, nan)","(nan, nan)","(157.03392857142856, 107.32499999999999)","(156.31378299120234, 115.13123167155425)","(155.76605504587155, 118.50573394495413)","(155.70033860045146, 118

In [9]:
samples, fails

(203, 218)

In [10]:
#df.to_csv('pixelcoordinates-complete.csv')